<a href="https://colab.research.google.com/github/KirteshVerma/Netflix-movies-and-TV-shows/blob/main/Netflix_movies_and_TV_shows_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import seaborn as sns
import pandas as pd

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:

path = '/content/drive/MyDrive/Captsone_Project_04/NETFLIX MOVIES AND TV SHOWS CLUSTERING.csv'

In [24]:
df = pd.read_csv(path)

In [25]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [26]:
df.shape

(7787, 12)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


In [28]:
# checking for duplicate values

duplicate_values = df.duplicated().sum()
print(f'The total number of duplicate values are {duplicate_values}')

The total number of duplicate values are 0


In [29]:
#checking null values
df.isnull().sum()

show_id            0
type               0
title              0
director        2389
cast             718
country          507
date_added        10
release_year       0
rating             7
duration           0
listed_in          0
description        0
dtype: int64

In [30]:
for i in df.columns:
    null_percentage = df[i].isna().sum() / len(df) * 100 
    if null_percentage > 0 :
        print("'{}' column null percentage :{}%".format(i,round(null_percentage,2)))

'director' column null percentage :30.68%
'cast' column null percentage :9.22%
'country' column null percentage :6.51%
'date_added' column null percentage :0.13%
'rating' column null percentage :0.09%


In [31]:
#lets drop null values in rating and date added as their are very few number of null values

df.dropna(subset=['rating', 'date_added'],inplace = True)

In [32]:
#filling the null values of country with its mode
df['country'] = df['country'].fillna(df['country'].mode()[0])

In [33]:
df['country'].value_counts()

United States                                                   3051
India                                                            923
United Kingdom                                                   396
Japan                                                            224
South Korea                                                      183
                                                                ... 
Russia, United States, China                                       1
Italy, Switzerland, France, Germany                                1
United States, United Kingdom, Canada                              1
United States, United Kingdom, Japan                               1
Sweden, Czech Republic, United Kingdom, Denmark, Netherlands       1
Name: country, Length: 681, dtype: int64

In [34]:
# as we can see country column has multiple countries in it so we will create a new column containing only one country 

df['prime_country'] = df['country'].apply(lambda x: x.split(",")[0])
df['prime_country'].head()

0           Brazil
1           Mexico
2        Singapore
3    United States
4    United States
Name: prime_country, dtype: object

In [35]:
df.dtypes

show_id          object
type             object
title            object
director         object
cast             object
country          object
date_added       object
release_year      int64
rating           object
duration         object
listed_in        object
description      object
prime_country    object
dtype: object

In [36]:
df['year_added'] =df['date_added'].apply(lambda x: x.split(" ")[-1])

In [37]:
df['month_added'] = df['date_added'].apply(lambda x: x.split(" ")[0])

In [38]:
#lets convert the category accourding to age wise for better understanding and thus making eda more interpretable 
age_wise_rating = {   'TV-PG': 'Older Kids',
    'TV-MA': 'Adults',
    'TV-Y7-FV': 'Older Kids',
    'TV-Y7': 'Older Kids',
    'TV-14': 'Teens',
    'R': 'Adults',
    'TV-Y': 'Kids',
    'NR': 'Adults',
    'PG-13': 'Teens',
    'TV-G': 'Kids',
    'PG': 'Older Kids',
    'G': 'Kids',
    'UR': 'Adults',
    'NC-17': 'Adults'}

In [39]:
df['age_wise_rating'] = df['rating'].replace(age_wise_rating)

In [40]:
df['age_wise_rating'].value_counts()

Adults        3617
Teens         2314
Older Kids    1327
Kids           512
Name: age_wise_rating, dtype: int64

In [41]:
df['type'] = pd.Categorical(df['type'])

In [42]:
df['age_wise_rating'] = pd.Categorical(df['age_wise_rating'], categories=['Kids', 'Older Kids', 'Teens', 'Adults' ])

In [43]:
df['year_added'] = pd.to_numeric(df['year_added'])

## EDA

In [44]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,prime_country,year_added,month_added,age_wise_rating
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,Brazil,2020,August,Adults
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,Mexico,2016,December,Adults
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow...",Singapore,2018,December,Adults
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi...",United States,2017,November,Teens
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,United States,2020,January,Teens
